In [1]:
!pip install numpy

You should consider upgrading via the 'D:\Practice flask\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install pandas

You should consider upgrading via the 'D:\Practice flask\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)


You should consider upgrading via the 'D:\Practice flask\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:

!pip install seaborn 


  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


You should consider upgrading via the 'D:\Practice flask\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
!!pip install nltk

['Collecting nltk',
 '  Downloading nltk-3.9.1-py3-none-any.whl (1.5 MB)',
 'Requirement already satisfied: click in d:\\practice flask\\.venv\\lib\\site-packages (from nltk) (8.1.7)',
 'Collecting tqdm',
 '  Downloading tqdm-4.67.1-py3-none-any.whl (78 kB)',
 'Collecting joblib',
 '  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)',
 'Collecting regex>=2021.8.3',
 '  Downloading regex-2024.11.6-cp310-cp310-win_amd64.whl (274 kB)',
 'Requirement already satisfied: colorama in d:\\practice flask\\.venv\\lib\\site-packages (from click->nltk) (0.4.6)',
 'Installing collected packages: tqdm, regex, joblib, nltk',
 'Successfully installed joblib-1.4.2 nltk-3.9.1 regex-2024.11.6 tqdm-4.67.1',
 'WARNING: You are using pip version 21.2.3; however, version 24.3.1 is available.',
 "You should consider upgrading via the 'D:\\Practice flask\\.venv\\Scripts\\python.exe -m pip install --upgrade pip' command."]

In [6]:
!pip install sklearn

Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'


    ERROR: Command errored out with exit status 1:
     command: 'D:\Practice flask\.venv\Scripts\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Natasha\\AppData\\Local\\Temp\\pip-install-wrw_1u2i\\sklearn_f17f87d0b29f4cf0bf694aab6f4f9c0a\\setup.py'"'"'; __file__='"'"'C:\\Users\\Natasha\\AppData\\Local\\Temp\\pip-install-wrw_1u2i\\sklearn_f17f87d0b29f4cf0bf694aab6f4f9c0a\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\Natasha\AppData\Local\Temp\pip-pip-egg-info-6u2ivw_c'
         cwd: C:\Users\Natasha\AppData\Local\Temp\pip-install-wrw_1u2i\sklearn_f17f87d0b29f4cf0bf694aab6f4f9c0a\
    Complete output (15 lines):
    The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
    rather than '

In [2]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
import re

import nltk
nltk.download('punkt')

from nltk import ngrams
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Natasha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Natasha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Natasha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
df = pd.read_csv('train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
df.shape

(7613, 5)

In [8]:
df.drop('id',axis=1,inplace=True)

In [9]:
df.duplicated().sum()

np.int64(52)

In [10]:
#remove duplicates
df = df.drop_duplicates(keep='first')

In [11]:
df.duplicated().sum()

np.int64(0)

In [12]:
df.shape

(7561, 4)

In [13]:
def clean_words(text):
    text = re.sub(r'(http.*)|#', '', text)
    tokens = word_tokenize(text)
    stops = stopwords.words('english')
    clean_tokens = [token.lower() for token in tokens if token.lower() not in stops and token.isalnum()]
    return ' '.join(clean_tokens)

In [14]:
nltk.download('punkt_tab')
df['text'] = df['text'].map(clean_words)
df.head()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Natasha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,keyword,location,text,target
0,NaN,NaN,deeds reason earthquake may allah forgive us,1
1,NaN,NaN,forest fire near la ronge sask canada,1
2,NaN,NaN,residents asked place notified officers evacua...,1
3,NaN,NaN,people receive wildfires evacuation orders cal...,1
4,NaN,NaN,got sent photo ruby alaska smoke wildfires pou...,1


In [15]:
df.isnull().sum()/len(df)*100

keyword      0.806772
location    33.064409
text         0.000000
target       0.000000
dtype: float64

In [16]:
df['keyword'].value_counts().sum()

np.int64(7500)

In [17]:
#lemmatizing using spacy
!pip install spacy
!python -m spacy download en_core_web_sm

import spacy

nlp = spacy.load("en_core_web_sm")

def lemmatize_text(text):

    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])

    return lemmatized_text

df['text'] = df['text'].apply(lemmatize_text)
df.head()

You should consider upgrading via the 'D:\Practice flask\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


You should consider upgrading via the 'd:\Practice flask\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


,keyword,location,text,target
0,NaN,NaN,deed reason earthquake may allah forgive we,1
1,NaN,NaN,forest fire near la ronge sask canada,1
2,NaN,NaN,resident ask place notify officer evacuation s...,1
3,NaN,NaN,people receive wildfire evacuation order calif...,1
4,NaN,NaN,get send photo ruby alaska smoke wildfire pour...,1


In [18]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Predefined list of disaster-related keywords
disaster_keywords = {"earthquake", "fire", "flood", "hurricane", "storm", "evacuation",
                     "tornado", "tsunami", "disaster", "wildfire", "explosion", "drought",
                     "landslide", "eruption", "pandemic", "emergency", "crisis", "sismo", "typhoon"}

# Function to extract disaster-related keyword and proper nouns from text
def extract_keywords(text):
    # Handle NaN values in 'text' by converting to string
    if not isinstance(text, str):
        text = str(text)

    # Tokenize the text and apply POS tagging
    words = word_tokenize(text.lower())  # Convert to lowercase for case-insensitive matching
    tagged_words = pos_tag(words)  # POS tagging

    # Extract disaster-related keywords
    disaster_matches = [word for word in words if word in disaster_keywords]

    # Extract proper nouns (NNP and NNPS)
    proper_nouns = [word for word, tag in tagged_words if tag in ['NNP', 'NNPS']]

    # Combine disaster-related keywords and proper nouns
    if disaster_matches:
        return ' '.join(disaster_matches)  # Return disaster-related keyword if found
    elif proper_nouns:
        return ' '.join(proper_nouns)  # Return proper nouns if no disaster keywords are found
    return None  # Return None if no keyword found

# Apply the keyword extraction only on rows where 'keyword' is NaN
df['keyword'] = df.apply(
    lambda row: extract_keywords(row['text']) if pd.isna(row['keyword']) else row['keyword'], axis=1
)

df.head()


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Natasha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Natasha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Natasha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,keyword,location,text,target
0,earthquake,NaN,deed reason earthquake may allah forgive we,1
1,fire,NaN,forest fire near la ronge sask canada,1
2,evacuation,NaN,resident ask place notify officer evacuation s...,1
3,wildfire evacuation,NaN,people receive wildfire evacuation order calif...,1
4,wildfire,NaN,get send photo ruby alaska smoke wildfire pour...,1


In [19]:
df.isnull().sum()

keyword       36
location    2500
text           0
target         0
dtype: int64

In [20]:
df.duplicated().sum()

np.int64(253)

In [21]:
# Check for duplicates based on specific columns
duplicate_rows = df[df.duplicated(subset=['text', 'keyword'])]


In [22]:
df = df.drop_duplicates()
df.duplicated().sum()


np.int64(0)

In [23]:
df.shape

(7308, 4)

In [24]:
df.head()

,keyword,location,text,target
0,earthquake,NaN,deed reason earthquake may allah forgive we,1
1,fire,NaN,forest fire near la ronge sask canada,1
2,evacuation,NaN,resident ask place notify officer evacuation s...,1
3,wildfire evacuation,NaN,people receive wildfire evacuation order calif...,1
4,wildfire,NaN,get send photo ruby alaska smoke wildfire pour...,1


In [25]:
nan_keyword_rows = df[df['keyword'].isna()]

# Display the filtered rows
print(nan_keyword_rows)

     keyword location                                               text  \
10      None      NaN                     three people die heat wave far   
14      None      NaN      damage school bus 80 multi car crash breaking   
15      None      NaN                                                man   
16      None      NaN                                         love fruit   
17      None      NaN                                      summer lovely   
18      None      NaN                                           car fast   
19      None      NaN                                    goooooooaaaaaal   
20      None      NaN                                         ridiculous   
21      None      NaN                                        london cool   
22      None      NaN                                        love skiing   
23      None      NaN                                      wonderful day   
24      None      NaN                                           looooool   
25      None

In [26]:
df['keyword'] = df['keyword'].fillna('unknown')

In [31]:
df.isnull().sum()

keyword        0
location    2345
text           0
target         0
dtype: int64

In [32]:
df.drop('location',axis=1,inplace=True)

In [33]:
df.head()

,keyword,text,target
0,earthquake,deed reason earthquake may allah forgive we,1
1,fire,forest fire near la ronge sask canada,1
2,evacuation,resident ask place notify officer evacuation s...,1
3,wildfire evacuation,people receive wildfire evacuation order calif...,1
4,wildfire,get send photo ruby alaska smoke wildfire pour...,1


In [34]:
# 1:real disaster
# 0:not real disaster

In [35]:
df.isnull().sum()

keyword    0
text       0
target     0
dtype: int64

In [37]:
#No null values found in dataframe

In [38]:
df['target'].value_counts()

target
0    4228
1    3080
Name: count, dtype: int64